In [38]:
import pymysql
from dotenv import load_dotenv
import os
from datetime import datetime, timedelta
import pandas as pd
from itertools import product

def DBMS_last_year_one_week_M2() :
    '''내일을 기준으로 작년 1주일치 생활인구수와 대여량을 DataFrame으로 반환하는 함수'''
    load_dotenv()
    host = os.getenv('host')
    user = os.getenv('user')
    password = os.getenv('password')
    database = os.getenv('database')
    port = int(os.getenv('port', 3306))

    # 현재 시각
    now = datetime.now()

    # 1주일치 시간 list
    this_week = [now + timedelta(days=i) for i in range(8)]

    # 1년 전 1주일치 데이터
    dt_1y = [day.replace(year=day.year - 1) for day in this_week]

    total_last_week = pd.DataFrame(columns=['연도','월','일','시','행정구','일시','총생활인구수_1년전','계절','출퇴근시간여부','주말구분','강수','기온','습도','풍속','대여량_1년전'])
    for data in dt_1y :
        query =f"""
            SELECT *
            FROM basic_data
            WHERE year = {data.year}
                AND month = {data.month}
                AND day = {data.day}
                """

        try:
            connection = pymysql.connect(
                host=host,
                user=user,
                password=password,
                database=database,
                port=port,
                charset='utf8mb4',
                cursorclass=pymysql.cursors.DictCursor
            )
            with connection.cursor() as cursor:
                cursor.execute(query)
                data = cursor.fetchall()
                whole_data = pd.DataFrame(data)
                whole_data.columns = ['연도','월','일','시','행정구','일시','총생활인구수_1년전','계절','출퇴근시간여부','주말구분','강수','기온','습도','풍속','대여량_1년전']
                total_last_week = pd.concat((total_last_week,whole_data),axis=0)
        except Exception as e:
            print('에러 발생:', e)
        finally:
            try:
                connection.close()
            except:
                pass
    total_last_week[['연도','월','일','시','행정구']] = total_last_week[['연도','월','일','시','행정구']].astype(str)
    return total_last_week

if __name__ == "__main__" :
    data = DBMS_last_year_one_week_M2()
    print(data)

       연도  월   일   시  행정구                  일시 총생활인구수_1년전  계절 출퇴근시간여부 주말구분  \
0    2024  7  10   0  강남구 2024-07-10 00:00:00     698456  여름      기타    0   
1    2024  7  10   0  강동구 2024-07-10 00:00:00     536265  여름      기타    0   
2    2024  7  10   0  강북구 2024-07-10 00:00:00     286823  여름      기타    0   
3    2024  7  10   0  강서구 2024-07-10 00:00:00     548005  여름      기타    0   
4    2024  7  10   0  관악구 2024-07-10 00:00:00     497139  여름      기타    0   
..    ... ..  ..  ..  ...                 ...        ...  ..     ...  ...   
592  2024  7  17  23  용산구 2024-07-17 23:00:00     238509  여름      기타    0   
593  2024  7  17  23  은평구 2024-07-17 23:00:00     463317  여름      기타    0   
594  2024  7  17  23  종로구 2024-07-17 23:00:00     201665  여름      기타    0   
595  2024  7  17  23   중구 2024-07-17 23:00:00     173282  여름      기타    0   
596  2024  7  17  23  중랑구 2024-07-17 23:00:00     359859  여름      기타    0   

      강수    기온    습도   풍속 대여량_1년전  
0    0.0  27.8  66.0  2.5      13  
1  

In [44]:
data['행정구'].unique()

array(['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구',
       '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구',
       '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구'], dtype=object)

In [45]:
import pandas as pd

dates = pd.date_range('2025-07-10', '2025-07-17')   # 8일
hours = list(range(24))
gus = data['행정구'].unique().tolist()

full = []
for dt in dates:
    for h in hours:
        for g in gus:
            full.append({
                '연도': str(dt.year),
                '월': str(dt.month),
                '일': str(dt.day),
                '시': str(h),
                '행정구': str(g)
            })
full_df = pd.DataFrame(full)
print(full_df.shape)  # (4800, 5)
full_df

(4800, 5)


,연도,월,일,시,행정구
0,2025,7,10,0,강남구
1,2025,7,10,0,강동구
2,2025,7,10,0,강북구
3,2025,7,10,0,강서구
4,2025,7,10,0,관악구
...,...,...,...,...,...
4795,2025,7,17,23,용산구
4796,2025,7,17,23,은평구
4797,2025,7,17,23,종로구
4798,2025,7,17,23,중구


In [48]:
merged = full_df.merge(data, on=['월','일','시','행정구'], how='left', indicator=True)
missing = merged[merged['_merge']=='left_only']
print(f"누락 행 개수: {len(missing)}")
print(missing[['월','일','시','행정구']])

누락 행 개수: 3
      월   일  시   행정구
4270  7  17  2   용산구
4438  7  17  9  서대문구
4445  7  17  9   용산구
